<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#load-data-and-transformation" data-toc-modified-id="load-data-and-transformation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>load data and transformation</a></span></li><li><span><a href="#define-a-simple-convolutional-neural-network" data-toc-modified-id="define-a-simple-convolutional-neural-network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>define a simple convolutional neural network</a></span></li><li><span><a href="#loss-function-and-optimizer" data-toc-modified-id="loss-function-and-optimizer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>loss function and optimizer</a></span></li><li><span><a href="#train" data-toc-modified-id="train-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>train</a></span></li><li><span><a href="#test" data-toc-modified-id="test-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>test</a></span></li><li><span><a href="#错误记录" data-toc-modified-id="错误记录-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>错误记录</a></span><ul class="toc-item"><li><span><a href="#RuntimeError:-error-executing-torch_shm_manager" data-toc-modified-id="RuntimeError:-error-executing-torch_shm_manager-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span><code>RuntimeError: error executing torch_shm_manager</code></a></span></li><li><span><a href="#Initializing-libiomp5.dylib,-but-found-libiomp5.dylib-already-initialized" data-toc-modified-id="Initializing-libiomp5.dylib,-but-found-libiomp5.dylib-already-initialized-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Initializing libiomp5.dylib, but found libiomp5.dylib already initialized</a></span></li></ul></li></ul></div>

CIFRA10数据集总共有60000张32*32的彩色图，其中5w张为训练集，1w张为测试集。样本属于10个类别，

# load data and transformation

In [1]:
import torch
import torchvision
import torchvision.transforms as T

In [2]:
mean, std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
file_root = '../../../../Workspace/Datasets/CIFAR10/'
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
batch_size = 32
num_workers = 2

In [3]:
def create_dataset(root, train=True, transform=None):
    return torchvision.datasets.CIFAR10(root=root, train=train, transform=transform)

In [4]:
transform_train = T.Compose([T.ToTensor(), 
                             T.Normalize(mean, std)])
trainset = create_dataset(file_root, True, transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [5]:
transform_test = T.Compose([T.ToTensor(), 
                            T.Normalize(mean, std)])

testset = create_dataset(file_root, False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# define a simple convolutional neural network

In [6]:
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(5 * 5 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 *5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# loss function and optimizer

In [8]:
from torch import optim

In [9]:
criterion = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# train

In [10]:
log_interval = 2000

In [11]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [12]:
for epoch in range(10):
    interval_loss = 0.0
    for x, (batch_images, batch_labels) in enumerate(train_loader):
        # clear the gradients of all optimized tensors
        opt.zero_grad()
        
        # forward
        outputs = net.forward(batch_images)
        # calculate loss
        loss = criterion(outputs, batch_labels)
        # BP
        loss.backward()
        opt.step()
        
        interval_loss += loss.item()
        if x % log_interval == (log_interval-1):
            print('epoch {}, iter {}, loss {%.6f}'.format(epoch+1, x+1, epoch_loss/log_interval))
            interval_loss = 0.0

        
print('Finished Training')

Finished Training


In [13]:
# save the model
path = './cifar_net.pth'
torch.save(net.state_dict(), path)

# test

In [14]:
# load back the model
net = Net()
net.load_state_dict(torch.load(path))

<All keys matched successfully>

In [15]:
correct = 0
total = 0
with torch.no_grad():
    for (batch_images, batch_labels) in test_loader:
        outputs = net(batch_images)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 58 %


# 错误记录

##  `RuntimeError: error executing torch_shm_manager`
torch1.3.0和torchvision0.4.0在MACOS上有问题，可以降低torch版本为1.2.0，或者重新安装torch

https://github.com/pytorch/pytorch/issues/2125


## Initializing libiomp5.dylib, but found libiomp5.dylib already initialized

解决方法1：根据后台的日志，可以设置环境变量，允许多个库存在
```
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
```

解决方法2：
安装nomkl库

https://stackoverflow.com/questions/53648730/omp-error-15-initializing-libiomp5-dylib-but-found-libiomp5-dylib-already-in